In [2]:
import os
import pandas as pd
import threading
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class TaskManager:
    def __init__(self, file_path, autosave_interval=60):
        self.file_path = file_path
        os.makedirs(os.path.dirname(self.file_path), exist_ok=True)

        try:
            self.tasks = pd.read_csv(file_path)
        except (FileNotFoundError, pd.errors.EmptyDataError):
            self.tasks = pd.DataFrame(columns=["description", "priority"])

        self.autosave_interval = autosave_interval
        self.autosave_running = True
        self.start_autosave()

    def save_tasks(self):
        try:
            self.tasks.to_csv(self.file_path, index=False)
        except PermissionError:
            print(f"Permission denied: Unable to write to {self.file_path}. Close the file if it is open and try again.")

    def autosave(self):
        while self.autosave_running:
            time.sleep(self.autosave_interval)
            self.save_tasks()
            print("Autosaved tasks.")

    def start_autosave(self):
        threading.Thread(target=self.autosave, daemon=True).start()

    def stop_autosave(self):
        self.autosave_running = False

    def add_task(self):
        description, priority = input("Description: "), input("Priority (Low, Medium, High): ")
        new_task = pd.DataFrame([{"description": description, "priority": priority}])
        self.tasks = pd.concat([self.tasks, new_task], ignore_index=True)
        self.save_tasks()
        print("Task added and saved successfully.")

    def remove_task(self):
        self.list_tasks()
        try:
            idx = int(input("Task number to remove: ")) - 1
            if 0 <= idx < len(self.tasks):
                self.tasks.drop(idx, inplace=True)
                self.tasks.reset_index(drop=True, inplace=True)
                self.save_tasks()
                print("Task removed and saved successfully.")
            else:
                print("Invalid task number.")
        except ValueError:
            print("Please enter a valid number.")

    def list_tasks(self):
        print(self.tasks if not self.tasks.empty else "No tasks available.")

    def recommend_task(self):
        if self.tasks.empty:
            print("No tasks available.")
            return

        desc = input("Describe task: ")
        vec = TfidfVectorizer()

        try:
            descriptions = self.tasks["description"].dropna().tolist()
            sim = cosine_similarity(vec.fit_transform(descriptions + [desc]))
            recommended_index = sim[-1][:-1].argmax()
            print("Recommendation:", self.tasks.iloc[recommended_index])
        except ValueError:
            print("Could not generate recommendations. Ensure tasks have meaningful descriptions.")

def main():
    file_path = r"C:\Users\Shaik Sofiya\Documents\8th sem\task1\basic\tasks.csv"
    tm = TaskManager(file_path, autosave_interval=30)  # Autosaves every 30 seconds

    actions = {
        "1": tm.add_task, "2": tm.remove_task, "3": tm.list_tasks, "4": tm.recommend_task,
        "5": tm.stop_autosave
    }

    while (choice := input("1:Add 2:Remove 3:List 4:Recommend 5:Exit: ")) != "5":
        actions.get(choice, lambda: print("Invalid option. Try again."))()

    tm.stop_autosave()  # Stop autosave when exiting
    tm.save_tasks()
    print("Tasks saved. Exiting...")

if __name__ == "__main__":
    main()


Task added and saved successfully.
Autosaved tasks.
                            description priority
0                         Buy groceries     High
1               Complete project report   Medium
2      Schedule a meeting with the team      Low
3  Prepare presentation for the meeting   Medium
4                         Pay the bills     High
5                              Exercise      Low
6                                  food     high
7                          reading book   medium
8                                            low
Autosaved tasks.
Task added and saved successfully.
Autosaved tasks.
                            description priority
0                         Buy groceries     High
1               Complete project report   Medium
2      Schedule a meeting with the team      Low
3  Prepare presentation for the meeting   Medium
4                         Pay the bills     High
5                              Exercise      Low
6                                  food     hi